# Fingerprints Quality Classification
## DNN Course JCE - Final Project 
### By Kobi Amsellem & Zohar Kedem

Expirements:
* Try to classify 5 classes that represat the quality of the finger print (0 poor, 4 best)

In [7]:
import numpy as np
import tensorflow as tf
import keras
import pathlib
import PIL

In [10]:
data_dir = pathlib.Path('./Dataset')
count = len(list(data_dir.glob('*/*.jpg')))

print('Total: ', count, ' examples')

Total:  976  examples


In [13]:
batch_size = 32
img_height = 710
img_width = 1080
seed=123
validation_split = 0.1


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=validation_split,
  subset="training",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  labels='inferred',
  label_mode='categorical',
  )

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=validation_split,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  labels='inferred',
  label_mode='categorical',
  )

Found 976 files belonging to 5 classes.
Using 879 files for training.
Found 976 files belonging to 5 classes.
Using 97 files for validation.


In [14]:

print(train_ds)
class_names = train_ds.class_names
print(class_names)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

<BatchDataset shapes: ((None, 710, 1080, 3), (None, 5)), types: (tf.float32, tf.float32)>
['0', '1', '2', '3', '4']
(32, 710, 1080, 3)
(32, 5)


In [35]:
# def preprocess_data(X,Y):
#     X_p=keras.applications.resnet50.preprocess_inpot(X)
#     Y_p=keras.utils.to_categorical(Y,5)
#     return X_p,Y_p





# print(train_ds)

In [36]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

8.7143395e-05 1.0


In [37]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [38]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [39]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [40]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
 1/25 [>.............................] - ETA: 1:14:28 - loss: 1.6252 - accuracy: 0.0938

KeyboardInterrupt: 